In [1]:
!pip install beautifulsoup4

In [2]:
!pip install selenium

In [3]:
#https://builtin.com/articles/selenium-web-scraping
#https://brightdata.com/blog/how-tos/using-selenium-for-web-scraping#:~:text=Before%20you%20start%20web%20scraping,Chrome%2C%20Firefox%2C%20and%20Edge.

#Import selenium and other required packages for the scrape
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import csv

In [4]:
#Set up the chrome driver options
options = Options()
options.add_argument("--headless")

#Set up the driver
service = Service("C:/Users/Jane Barraclough/chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

In [5]:
#Get the url
url = "https://howstat.com/Cricket/Statistics/Matches/MatchListCountry.asp?A=AUS&B=XXX&C=XXX"
driver.get(url)

#Get the full HTML and close the browser
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
driver.quit()

In [6]:
#I know I need table 5 for my analysis from the previous scrape attempt
tables = soup.find_all('table')
match_table = tables[5]
rows = match_table.find_all('tr')

In [7]:
#Need to confirm the number of rows matches the matches played
print(len(rows))

874


In [8]:
#Get the column names
header_row = rows[0]
headers = [cell.text.strip() for cell in header_row.find_all(['th', 'td'])]
print(headers)

['', 'Date', 'Series', 'Ground', 'Result']


In [9]:
#Extract all match data rows
data_rows = []

for row in rows[1:]:
    cells = [cell.text.strip() for cell in row.find_all(['th', 'td'])]
    if cells:
        data_rows.append(cells)

print(data_rows[0:4])

[['1', '15/03/1877', '1876-1877 Australia v. England', 'Melbourne Cricket Ground', 'Australia won by 45 runs'], ['2', '31/03/1877', '1876-1877 Australia v. England', 'Melbourne Cricket Ground', 'England won by 4 wickets'], ['3', '02/01/1879', '1878-1879 Australia v. England', 'Melbourne Cricket Ground', 'Australia won by 10 wickets'], ['4', '06/09/1880', '1880 England v. Australia', 'Kennington Oval', 'England won by 5 wickets']]


In [10]:
df = pd.DataFrame(data_rows, columns=headers)
df.columns = ['Test Number', 'Date', 'Series', 'Ground', 'Result']
df.head()

,Test Number,Date,Series,Ground,Result
0,1,15/03/1877,1876-1877 Australia v. England,Melbourne Cricket Ground,Australia won by 45 runs
1,2,31/03/1877,1876-1877 Australia v. England,Melbourne Cricket Ground,England won by 4 wickets
2,3,02/01/1879,1878-1879 Australia v. England,Melbourne Cricket Ground,Australia won by 10 wickets
3,4,06/09/1880,1880 England v. Australia,Kennington Oval,England won by 5 wickets
4,5,31/12/1881,1881-1882 Australia v. England,Melbourne Cricket Ground,Match Drawn


In [11]:
#Download to a CSV file
df.to_csv('../data/australia_match_results.csv', index=False)